In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Installing the necessary files
!pip install wikipedia
!python -m textblob.download_corpora

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=bdc96d811072577e21bf823c80a2b593f64cda3be9d380bbec214a2d46f09cbf
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [0]:
#importing other necessary libraries
import wikipedia as wki
from textblob import TextBlob as txb
import pandas as pd
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math
import statistics
from operator import itemgetter
import random
from sklearn.metrics.pairwise import cosine_similarity as cs
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import numpy as np
from difflib import SequenceMatcher

In [0]:
# Get the WIKI dump of FEVER
table = []
nums = [f"{i:03}" for i in range(1,110)]
for i in nums:
    with open('/content/drive/My Drive/NLP/wiki-pages/wiki-'+i+'.jsonl') as f:
        for line in f:
          table.append(json.loads(line))

In [0]:
#Get the test Claims that needs to be labelled
# test_claims = []
# with open('/content/drive/My Drive/NLP/shared_task_dev_public.jsonl', 'r') as f:
#     for line in f:
#         test_claims.append(json.loads(line))

In [7]:
# This libraries will help to deploy model to web app
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify, render_template
# from flask_debug import Debug
import pickle

In [8]:
!pip install transformers
!pip install jsonlines

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 1.0MB 16.8MB/s 
     |████████████████████████████████| 3.7MB 22.5MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=691230603439af43f1183a86116b98dfbc1075dc4ea13ec076d7ae5fbe19bcb3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

In [11]:
# Load pre-trained model and tokenizer 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
#Code for running the Web app
import numpy as np
import jsonlines
import re
# app = Flask(__name__)
app = Flask(__name__, root_path='/content/drive/My Drive/')
run_with_ngrok(app)
def remove_string_special_characters(s):
    stripped =  re.sub('[^\w\s]', '', s)
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    stripped = stripped.strip()
    return stripped


def get_doc(sents):
    doc_info = []
    i = 0
    for sent in sents:
        i += 1
        count = count_words(sent)
        temp = {'doc_id' : i, 'doc_length': count}
        doc_info.append(temp)
    return doc_info


def count_words(sent):
    count = 0
    words = word_tokenize(sent)
    for word in words:
        count += 1
    return count

def create_freq_dict(sents):
    i = 0
    freqDict_list = []
    for sent in sents:
        i += 1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
            temp = {'doc_id' : i, 'freq_dict': freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

def computeTF(doc_info, freqDict_list):
    TF_scores = []
    for tempDict in freqDict_list:
        id = tempDict['doc_id']
        for k in tempDict['freq_dict']:
            temp = {'doc_id' : id, 'TF_score' : tempDict['freq_dict'][k]/doc_info[id-1]['doc_length'], 'key' : k}
            TF_scores.append(temp)
    return TF_scores


def computeIDF(doc_info, freqDict_list):
    IDF_scores = []
    counter = 0
    for dict in freqDict_list:
        counter += 1
        for k in dict['freq_dict'].keys():
            count = sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp = {'doc_id' : counter, 'IDF_score' : math.log(len(doc_info)/count), 'key' : k}
            IDF_scores.append(temp)
            
    return IDF_scores


def computeTFIDF(TF_scores, IDF_scores):
    TFIDF_scores = []
    for j in IDF_scores:
        for i in TF_scores:
            if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                temp = {'doc_id' : j['doc_id'], 'TFIDF_score' : j['IDF_score'] * i['TF_score'], 'key' : i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

def get_sent_score(TFIDF_scores, text_sents, doc_info):
    sentence_info = []
    for doc in doc_info:
        sent_score = 0
        for i in range(0, len(TFIDF_scores)):
            temp_dict = TFIDF_scores[i]
            if doc['doc_id'] == temp_dict['doc_id']:
                sent_score += temp_dict['TFIDF_score']
        temp = {'doc_id' : doc['doc_id'], 'sent_score' : sent_score, 'sentence' : text_sents[doc['doc_id']-1]}
        sentence_info.append(temp)
    return sentence_info

def call_TFIDF(lines, claim):
    text_sents = sent_tokenize(lines)
    text_sents_clean = [remove_string_special_characters(s) for s in text_sents]
    doc_info = get_doc(text_sents_clean)
    freqDict_list = create_freq_dict(text_sents_clean)
    TF_scores = computeTF(doc_info,freqDict_list)
    IDF_scores = computeIDF(doc_info, freqDict_list)
    TFIDF_scores = computeTFIDF(TF_scores, IDF_scores)
    sentence_info = get_sent_score(TFIDF_scores, text_sents, doc_info)
    #sort_sent = (sorted(sentence_info, key = lambda i: i['sent_score'],reverse=True))
    #return sort_sent[0]['sentence'], sort_sent[0]['doc_id']-1
    for sent in sentence_info:
        tfidf = TfidfVectorizer(norm='l2', min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, stop_words='english')
        #print(sent['sentence'])
        text = str(sent['sentence'])
        #print(text + "---")
        #print(claim)
        #print(sent['sentence'][0])
        X = tfidf.fit_transform([text] + [claim])
        X_sparse = sparse.csr_matrix(X)
        #print(X_sparse)
        similarity = cs(X_sparse)
        add = similarity[0][1]
        #print(claim)
        #print(add)
        #print(sent['sent_score'])
        sent['sent_score'] = sent['sent_score'] + add
        #print(sent['sent_score'])
    sort_sent = (sorted(sentence_info, key = lambda i: i['sent_score'],reverse=True))
    return sort_sent[0]['sentence'], sort_sent[0]['doc_id']-1





#define example and feature classes
class InputExample(object):
  def __init__(self, claim, ev, label=None):
    self.claim = claim
    self.ev = ev
    self.label = label

class InputFeatures(object):
    def __init__(self, claim_ids, ev_ids, label_id):
        self.claim_ids = claim_ids
        self.ev_ids = ev_ids
        self.label_id = label_id

def data_to_examples(line):
  examples = []
  #data_file=json.dumps(data1)
  #data = jsonlines.open(data_file)
  #labels = jsonlines.open('train_output.jsonl')
  #for line,line2 in zip(data,labels):
  #for index in range(845):
  #line = data.read()
# label = line2['label']
  claim = line['claim']
  ev=[]
  if line['evidences'] != 'null':
    for item in line['evidences']:
      #remove nonsense from sents
      clean = re.sub('\n+',' ',item)
      clean = re.sub('\s+',' ',clean)
      clean = re.sub('[a-zA-Z0-9]*\t[a-zA-Z0-9]*','',clean)
      clean = re.sub('-...-','',clean)
      if len(clean.split()) <= 512:
        ev.append(clean)
  else:
    ev.append('null')
  examples.append(InputExample(claim,ev))#,label))
  return examples

def example_to_features(example):
  #tokenize and add [CLS]/[SEP] tokens
  claim_tokens = tokenizer.encode(example.claim, add_special_tokens=True)
  ev_tokens = []
  if example.ev ==[]:
    ev_tokens.append([101])
  for item in example.ev:
    ev_tokens.append(tokenizer.encode(item,add_special_tokens=True))
  
  return InputFeatures(claim_tokens,ev_tokens,example.label)


loaded_model = pickle.load(open('/content/drive/My Drive/NLP/new_trained_model_15k.sav', 'rb'))      


#print(type(lines))

@app.route('/')
def home():
  return render_template('index.html')

@app.route('/test',methods=['POST'])
def test():
  # claim = request.form["claim_text_area"]
  
  # claim = [request.form.values()]

  claim=request.form['claim']
  result={}
  #counter += 1
  id_row = []
  concat_string = []
  phrases=[]   
  #temp_str=claims['claim']
  #rake.extract_keywords_from_text(temp_str)
  blob = txb(claim)
  phrases = blob.noun_phrases
  #print(phrases)
  if(len(phrases) != 0):
    results = wki.search(phrases, results=3)
     #results = claim['results']
     #print("RESULTS")
  for result in results:
    if result is not 'null':
      temp = result.replace(" ", "_")
      try:
        lines = str(next(item['lines'] for item in table if item["id"] == temp))
      except StopIteration:
        continue
      try:
        temp_sent, sent_id = call_TFIDF(lines, claim)
      except IndexError:
        continue
      concat_string.append(temp_sent)
      temp_id = []
      temp_id.append(result)
      temp_id.append(sent_id)
      id_row.append(temp_id)
  output_row = {}
  output_row['claim'] = claim
  output_row['sources'] = id_row
  output_row['evidences'] = concat_string
  # json.dump(output_row, outfile);
  # outfile.write("\n");
  # print(counter)
  #return output_row

  test_ex = data_to_examples(output_row)

  feat_list = []
  for example in test_ex:
    features = example_to_features(example)
    feat_list.append(features)


  #####BOTTLENECK STEP
#create BERT representations for each example
  max_len = 432
  BERT_examples = []
  round = 0
  for item in feat_list:
    #padding
    padded_claim = np.array(item.claim_ids + [0]*(max_len-len(item.claim_ids)))
    padded_claim = np.reshape(padded_claim,(1,max_len))
    padded_ev = np.array([sent + [0]*(max_len-len(sent)) for sent in item.ev_ids])
    
    #mask padded tokens
    claim_mask = np.where(padded_claim != 0,1,0)
    ev_mask = np.where(padded_ev != 0,1,0)

    #create matrix: each row represents one sentence
    padded_total = np.concatenate((padded_claim,padded_ev),axis=0)
    mask_total = np.concatenate((claim_mask,ev_mask))

    #convert to tensors
    input_ids = torch.tensor(padded_total)
    attention_mask = torch.tensor(mask_total)

    #get BERT embeddings
    with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)

    #extract [CLS] embeddings and labels for input to classifier
    features = last_hidden_states[0][:,0,:].numpy()
    concatenated = features.flatten()
    BERT_examples.append(list(concatenated))
    round += 1
  
  #pad each example for uniform input size
  max_len = 109824

  padded_BERT = []
  for example in BERT_examples:
    padded_BERT.append(example + [0]*(max_len-len(example)))
  
  test_inputs = np.array([np.array(ex) for ex in padded_BERT])
  
  predictions = loaded_model.predict(test_inputs)
  answer="The claim \""+claim+"\""
  label_color="red"
  if predictions[0]=="SUPPORTS":
    result1="IS TRUE (SUPPORTS)"
  elif predictions[0]=="REFUTES":
    result1="IS FALSE (REFUTES)"
  else:
    result1="Cannot be determined"

  # if eachSource:
  #   for i in range(len(eachSource)):
  #     if(i==0):
  #       source_text0=eachSource[0]
  #     elif(i==1):
  #       source_text1=eachSource[1]
  #     elif(i==2):
  #       source_text2=eachSource[2]  

  #sourceEvidence = remove_string_special_characters(s)
  #tohit=np.array(claim)
  return render_template('index.html', result_text="{}".format(result1),claim_text='{}'.format(answer))
  # return render_template('index.html',source_text0="{}".format(source_text0),source_text1="{}".format(source_text1),source_text2="{}".format(source_text2), result_text="{}".format(result1),claim_text='{}'.format(answer))

if __name__ == "__main__":
  # Debug(app)
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0876f781.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/May/2020 15:29:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 15:29:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/May/2020 15:29:26] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2020 15:29:52] "POST /test HTTP/1.1" 200 -
